In [1]:
#Standard python libraries
import os
import time
import logging
import pickle
 
# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import torch
# Imports from our package
import lightautoml
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ImportanceCutoffSelector, ModelBasedImportanceEstimator
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
#from lightautoml.utils.profiler import Profiler
from lightautoml.automl.blend import WeightedBlender
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.report import ReportDeco

In [2]:
df = pd.read_csv(r"C:\Users\kaush\Documents\6390_Kaggle_Project_2\new_train.csv")

In [3]:
df = df.drop(columns=["Unnamed: 0"])
df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887,0.0
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779,1.0
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492,1.0
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528,1.0
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910,0.0
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089,1.0
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659,0.0
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107,1.0


In [4]:
df.describe()

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
count,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,...,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000
mean,-11.795119,-18.078254,2081.046712,2100.501195,2304.266747,2667.026142,3009.954514,3153.526957,3219.598875,3318.671884,...,3354.115868,1292.793264,2070.074244,1295.342055,280.991866,0.003141,502.305559,4.679795,0.259815,0.676976
std,4.208928,4.720095,1934.856503,1933.397044,2085.701681,2339.775198,1972.864966,1919.396478,1926.329600,1891.359026,...,1894.715573,1211.689623,1166.978098,839.315313,14.017163,0.003583,586.159308,6.390935,0.196771,0.467636
min,-57.171004,-74.840663,-56.392936,299.238220,308.116962,136.928851,169.569203,84.967215,88.123196,13.675767,...,190.315943,30.882181,49.438014,47.248169,240.090890,0.000091,-119.033169,-0.163469,-0.643869,0.000000
25%,-13.911653,-20.583776,951.073058,949.720040,992.328940,1251.564736,1782.774981,1885.205452,1950.921249,2040.184965,...,2059.526734,551.741199,1276.246327,748.817994,269.644897,0.000978,124.780914,1.320973,0.100172,0.000000
50%,-11.504172,-17.643541,1163.268277,1210.112302,1437.884700,1648.392700,2302.365800,2523.560200,2597.782990,2761.135720,...,2833.860815,802.781409,1879.542349,1053.972349,278.290841,0.001794,293.217361,2.470939,0.240179,1.000000
75%,-9.322609,-14.926473,2103.385354,2156.902557,2565.968373,2921.320072,3351.444404,3735.071878,3802.460633,4015.688122,...,4084.349790,1457.314397,2650.952606,1646.218963,294.491275,0.003582,692.439658,4.868427,0.401444,1.000000
max,14.057914,7.074079,13624.663133,14274.620618,15724.538745,18868.088113,17166.621988,16868.053089,17286.252694,16836.840939,...,19489.963954,13943.846879,13135.748022,11376.935402,305.419884,0.045823,5376.089584,73.877508,0.796425,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59624 entries, 0 to 59623
Columns: 217 entries, S2_B2_jan to LABELS
dtypes: float64(217)
memory usage: 98.7 MB


In [6]:
df.dtypes

S2_B2_jan                       float64
S2_B3_jan                       float64
S2_B4_jan                       float64
S2_B5_jan                       float64
S2_B6_jan                       float64
                                 ...   
ERA5_total_precipitation_dec    float64
topo_elevation_dec              float64
topo_slope_dec                  float64
NDVI_dec                        float64
LABELS                          float64
Length: 217, dtype: object

In [7]:
labels = df[["LABELS"]]

In [8]:
labels.value_counts()

LABELS
1.0       40364
0.0       19260
dtype: int64

In [9]:
# There is some imbalance in the data. But not much.

In [10]:
df.isna().sum()

S2_B2_jan                       0
S2_B3_jan                       0
S2_B4_jan                       0
S2_B5_jan                       0
S2_B6_jan                       0
                               ..
ERA5_total_precipitation_dec    0
topo_elevation_dec              0
topo_slope_dec                  0
NDVI_dec                        0
LABELS                          0
Length: 217, dtype: int64

In [11]:
df[["LABELS"]] = df[["LABELS"]].astype(int)
df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887,0
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779,1
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492,1
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528,1
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910,0
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089,1
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659,0
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107,1


In [12]:
X = df.drop(columns=["LABELS"])
X

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,1939.707676,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2816.650882,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2425.676203,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2233.688637,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,3569.602117,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2487.672188,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,6940.383810,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2364.680153,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4080.569025,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107


In [13]:
y = df[["LABELS"]]

# Scaling features:

# We know from our previous accuracy that this model already does pretty well, so let's just train on the full dataset.

In [14]:
scaler = preprocessing.StandardScaler()

In [15]:
X_full_scaled = scaler.fit_transform(X)

In [16]:
# X_test_scaled = scaler.transform(X_test)
test = pd.read_csv(r"C:\Users\kaush\Documents\6390_Kaggle_Project_2\test_nolabels.csv")
test = test.drop(columns=["S.No"])

In [17]:
X_test_submission= scaler.transform(test)

In [18]:
X_test_submission.shape

(1200, 216)

In [19]:
X_full_scaled_df = pd.DataFrame(X_full_scaled)
X_full_scaled_df

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-0.588316,-0.647433,-0.580676,-0.559095,-0.515056,-0.526503,-0.667184,-0.702370,-0.689421,-0.686340,...,-0.620544,-0.607532,-0.410117,0.580739,0.469024,0.598582,-0.835673,2.361287,-0.615037,0.244307
1,-0.129799,0.706194,-0.584968,-0.623082,-0.631327,-0.562678,-0.547287,-0.575623,-0.438100,-0.559797,...,-0.179512,-0.227864,-0.125443,0.392913,0.761246,-1.028283,-0.074048,-0.737740,-0.515405,-0.823481
2,-0.124196,0.222890,-0.513443,-0.524522,-0.536280,-0.486099,-0.583407,-0.650658,-0.590062,-0.636034,...,-0.376141,-0.425075,-0.636877,-0.440451,-0.660221,-0.322749,-0.424135,-0.701224,-0.399614,0.806414
3,-0.840449,-0.614389,0.468348,0.549841,0.531387,0.530576,0.524256,0.561549,0.240413,0.634581,...,-0.472695,-0.430976,-0.180910,0.272558,0.086190,-0.348866,0.161474,0.452972,0.586448,0.389866
4,0.013214,1.420321,-0.718003,-0.681909,-0.875405,-0.722414,0.090823,0.344052,0.333396,0.452546,...,0.199161,0.486719,-0.760045,0.009055,-0.717986,1.260706,-0.384760,-0.752028,-0.543884,1.705496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,0.862865,0.601903,-0.564941,-0.490980,-0.508596,-0.468716,-0.012519,0.141257,0.147961,0.247173,...,-0.344962,-0.267699,-0.616485,2.406115,1.958449,0.898329,1.530850,0.462498,-0.145409,0.239344
59620,0.090841,-0.098899,-0.061886,-0.101382,-0.007983,0.054656,-0.071715,-0.104632,-0.128333,-0.062961,...,1.894392,2.082113,0.639668,3.493864,3.546415,-1.069486,-0.024856,-0.523411,-0.472310,-0.933708
59621,-1.891840,-0.189854,-0.368010,-0.208715,0.035388,-0.102732,-0.252813,-0.219718,-0.281888,-0.212841,...,-0.406817,-0.347371,-0.644218,1.645694,1.641309,1.306827,-0.517646,0.019553,-0.169182,0.075441
59622,0.299063,0.570189,0.486467,0.655626,0.686877,0.570040,0.921570,1.140529,0.985343,1.299968,...,0.456136,0.569833,-0.322023,-0.257184,-0.756496,0.837075,2.354249,-0.010612,1.229572,1.866611


In [20]:
X_full_scaled_df.columns = df.columns[:-1]

In [21]:
X_full_scaled_df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-0.588316,-0.647433,-0.580676,-0.559095,-0.515056,-0.526503,-0.667184,-0.702370,-0.689421,-0.686340,...,-0.620544,-0.607532,-0.410117,0.580739,0.469024,0.598582,-0.835673,2.361287,-0.615037,0.244307
1,-0.129799,0.706194,-0.584968,-0.623082,-0.631327,-0.562678,-0.547287,-0.575623,-0.438100,-0.559797,...,-0.179512,-0.227864,-0.125443,0.392913,0.761246,-1.028283,-0.074048,-0.737740,-0.515405,-0.823481
2,-0.124196,0.222890,-0.513443,-0.524522,-0.536280,-0.486099,-0.583407,-0.650658,-0.590062,-0.636034,...,-0.376141,-0.425075,-0.636877,-0.440451,-0.660221,-0.322749,-0.424135,-0.701224,-0.399614,0.806414
3,-0.840449,-0.614389,0.468348,0.549841,0.531387,0.530576,0.524256,0.561549,0.240413,0.634581,...,-0.472695,-0.430976,-0.180910,0.272558,0.086190,-0.348866,0.161474,0.452972,0.586448,0.389866
4,0.013214,1.420321,-0.718003,-0.681909,-0.875405,-0.722414,0.090823,0.344052,0.333396,0.452546,...,0.199161,0.486719,-0.760045,0.009055,-0.717986,1.260706,-0.384760,-0.752028,-0.543884,1.705496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,0.862865,0.601903,-0.564941,-0.490980,-0.508596,-0.468716,-0.012519,0.141257,0.147961,0.247173,...,-0.344962,-0.267699,-0.616485,2.406115,1.958449,0.898329,1.530850,0.462498,-0.145409,0.239344
59620,0.090841,-0.098899,-0.061886,-0.101382,-0.007983,0.054656,-0.071715,-0.104632,-0.128333,-0.062961,...,1.894392,2.082113,0.639668,3.493864,3.546415,-1.069486,-0.024856,-0.523411,-0.472310,-0.933708
59621,-1.891840,-0.189854,-0.368010,-0.208715,0.035388,-0.102732,-0.252813,-0.219718,-0.281888,-0.212841,...,-0.406817,-0.347371,-0.644218,1.645694,1.641309,1.306827,-0.517646,0.019553,-0.169182,0.075441
59622,0.299063,0.570189,0.486467,0.655626,0.686877,0.570040,0.921570,1.140529,0.985343,1.299968,...,0.456136,0.569833,-0.322023,-0.257184,-0.756496,0.837075,2.354249,-0.010612,1.229572,1.866611


In [22]:
X_test_submission_df = pd.DataFrame(X_test_submission)

In [23]:
X_test_submission_df

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-1.390950,-1.323909,-0.501046,-0.470855,-0.451845,-0.526033,-0.841259,-0.922910,-0.934366,-1.008921,...,-1.204898,-1.260148,-0.680924,-0.878104,-0.788210,1.138654,-0.851177,-0.385289,-0.612406,-1.317349
1,-0.656840,-0.393611,-0.163451,-0.126151,-0.070733,-0.014407,-0.140944,-0.143163,-0.310580,-0.061924,...,-0.576793,-0.560319,-0.413380,0.712946,0.516595,1.246815,-0.837873,-0.537700,-0.325660,-0.216077
2,-0.573482,-0.704666,-0.450025,-0.441958,-0.374331,-0.410459,-0.504646,-0.497547,-0.489108,-0.407842,...,-0.538070,-0.519500,-0.329364,0.982832,1.028549,1.216601,-0.847858,-0.302733,-0.613529,-0.148782
3,-0.499013,-0.654915,-0.274075,-0.289215,-0.274208,-0.358310,-0.574377,-0.600466,-0.634694,-0.584691,...,-0.539076,-0.518025,-0.372595,1.019303,0.815613,1.251505,-0.840383,-0.542463,-0.554980,-0.228068
4,0.055231,0.949849,-0.470052,-0.495108,-0.483220,-0.551403,-0.771027,-0.845341,-0.805783,-0.836221,...,-0.627081,-0.653269,-0.366070,0.412060,0.192659,1.241158,-0.845178,-0.410691,-0.613359,-0.199496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,-0.749046,-0.157601,-0.460515,-0.417189,-0.321270,-0.361598,-0.516184,-0.524418,-0.491765,-0.472669,...,-0.749283,-0.768350,-0.626273,0.360089,0.378412,1.229432,-0.847170,-0.267805,-0.306111,-0.406629
1196,0.648804,0.958982,-0.502476,-0.503881,-0.538587,-0.496435,-0.508158,-0.513771,-0.452978,-0.491340,...,-0.652729,-0.671957,-0.284502,0.196881,0.078262,1.189010,-0.851172,-0.455144,-0.329732,-0.013886
1197,1.205028,1.193331,-0.483880,-0.470339,-0.425545,-0.485629,-0.577387,-0.614155,-0.658604,-0.638109,...,-0.840808,-0.880480,-0.716814,0.056467,0.190394,1.165143,-0.851177,-0.278919,0.654321,-0.623459
1198,0.463372,0.770895,-0.544437,-0.559611,-0.619331,-0.591337,-0.502138,-0.506673,-0.465198,-0.391246,...,-0.710561,-0.653269,-0.595277,-0.234389,-0.548090,1.244745,-0.843138,-0.432917,0.399669,0.190296


In [24]:
X_test_submission_df.columns = df.columns[:-1]

In [25]:
data = pd.concat([X_full_scaled_df, y], axis = 1)

In [38]:
data

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-0.588316,-0.647433,-0.580676,-0.559095,-0.515056,-0.526503,-0.667184,-0.702370,-0.689421,-0.686340,...,-0.607532,-0.410117,0.580739,0.469024,0.598582,-0.835673,2.361287,-0.615037,0.244307,0
1,-0.129799,0.706194,-0.584968,-0.623082,-0.631327,-0.562678,-0.547287,-0.575623,-0.438100,-0.559797,...,-0.227864,-0.125443,0.392913,0.761246,-1.028283,-0.074048,-0.737740,-0.515405,-0.823481,1
2,-0.124196,0.222890,-0.513443,-0.524522,-0.536280,-0.486099,-0.583407,-0.650658,-0.590062,-0.636034,...,-0.425075,-0.636877,-0.440451,-0.660221,-0.322749,-0.424135,-0.701224,-0.399614,0.806414,1
3,-0.840449,-0.614389,0.468348,0.549841,0.531387,0.530576,0.524256,0.561549,0.240413,0.634581,...,-0.430976,-0.180910,0.272558,0.086190,-0.348866,0.161474,0.452972,0.586448,0.389866,1
4,0.013214,1.420321,-0.718003,-0.681909,-0.875405,-0.722414,0.090823,0.344052,0.333396,0.452546,...,0.486719,-0.760045,0.009055,-0.717986,1.260706,-0.384760,-0.752028,-0.543884,1.705496,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,0.862865,0.601903,-0.564941,-0.490980,-0.508596,-0.468716,-0.012519,0.141257,0.147961,0.247173,...,-0.267699,-0.616485,2.406115,1.958449,0.898329,1.530850,0.462498,-0.145409,0.239344,0
59620,0.090841,-0.098899,-0.061886,-0.101382,-0.007983,0.054656,-0.071715,-0.104632,-0.128333,-0.062961,...,2.082113,0.639668,3.493864,3.546415,-1.069486,-0.024856,-0.523411,-0.472310,-0.933708,1
59621,-1.891840,-0.189854,-0.368010,-0.208715,0.035388,-0.102732,-0.252813,-0.219718,-0.281888,-0.212841,...,-0.347371,-0.644218,1.645694,1.641309,1.306827,-0.517646,0.019553,-0.169182,0.075441,0
59622,0.299063,0.570189,0.486467,0.655626,0.686877,0.570040,0.921570,1.140529,0.985343,1.299968,...,0.569833,-0.322023,-0.257184,-0.756496,0.837075,2.354249,-0.010612,1.229572,1.866611,1


In [27]:
#define all the parameters
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.15 # Test size for metric check
TARGET_NAME = 'LABELS' # Target column name

In [37]:
# train_data, test_data = train_test_split(data, 
#                                          test_size=TEST_SIZE, 
#                                          stratify=data[TARGET_NAME],                                          
#                                          random_state=RANDOM_STATE)

In [33]:
#Referred to: https://towardsdatascience.com/lightautoml-preset-usage-tutorial-2cce7da6f936
def f1(y_true, y_pred):
    return f1_score(y_true, (y_pred > 0.5).astype(int))

In [34]:
#create task for binary classification
task = Task('binary', metric=f1)

In [36]:
#AutoML can do this automatically for all other variables except for target variables.
roles = {'target': TARGET_NAME,
         }

In [ ]:
RD = ReportDeco(output_path = 'tabularAutoML_model_report')
TIMEOUT = 2700
automl_rd = RD(TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       general_params = {'nested_cv': False, 'use_algos': [['lgb', 'lgb_tuned', 'catb', 'catb_tuned']]},
                       reader_params = {'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                       tuning_params = {'max_tuning_iter': 20, 'max_tuning_time': 75},
                       lgb_params = {'default_params': {'num_threads': N_THREADS}}))
oof_pred = automl_rd.fit_predict(data, roles = roles, verbose = 1)

[14:26:26] Start automl utilizator with listed constraints:
[14:26:26] - time: 2700.00 seconds
[14:26:26] - CPU: 4 cores
[14:26:26] - memory: 16 GB

[14:26:26] If one preset completes earlier, next preset configuration will be started

[14:26:26] ==================================================
[14:26:26] Start 0 automl preset configuration:
[14:26:26] C:\Users\kaush\anaconda3\envs\ift6758-conda-env\lib\site-packages\lightautoml\automl\presets\tabular_configs\conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[14:26:26] Stdout logging level is INFO.
[14:26:26] Task: binary

[14:26:26] Start automl preset with listed constraints:
[14:26:26] - time: 2700.00 seconds
[14:26:26] - CPU: 4 cores
[14:26:26] - memory: 16 GB

[14:26:26] Train data shape: (59624, 217)

[14:26:42] Layer 1 train process start. Time left 2683.94 secs
[14:26:50] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[14:31:40] Fitting Lvl_0_Pipe_

In [ ]:
!ls tabularAutoML_model_report

In [ ]:
# Fast feature importances calculation
fast_fi = automl_rd.model.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

In [ ]:
# Accurate feature importances calculation (Permutation importances) -  can take long time to calculate
accurate_fi = automl_rd.model.get_feature_scores('accurate', test_data, silent = False)